<a href="https://colab.research.google.com/github/susanna-susanna/dau-stpy/blob/main/stpy_1_%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B5_%D0%BF%D0%BE%D0%BD%D1%8F%D1%82%D0%B8%D1%8F_%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8_%D0%B2%D1%8B%D0%B1%D1%80%D0%BE%D1%81%D1%8B%2C_%D0%BF%D1%83%D1%81%D1%82%D0%BE%D1%82%D1%8B%2C_IQR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Домашнее задание к лекции "Базовые понятия статистики"
Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

###Задание 1. Загрузка данных
Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names), загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

###Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

###Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

Информация об атрибутах:

  1: операция?

         1. 1 = да, была операция
         2. 2 = лечили без операции

  2: Возраст

          1. 1 = взрослая лошадь
          2. 2 = Молодой (< 6 месяцев)

  
  4: ректальная температура

          * линейный
          * в градусах Цельсия.
          * Повышенная температура может возникнуть из-за инфекции.
          * температура может быть снижена, когда животное находится в позднем шоке
          * нормальная температура 37,8
          * этот параметр обычно меняется по мере развития проблемы,например, может начаться нормально, затем стать повышенным из-за поражения, проходя обратно через нормальный диапазон, как лошадь в шоке
  
  5: пульс

          * линейный
          * частота сердечных сокращений в ударах в минуту
          * является отражением состояния сердца: 30-40 нормально для взрослых
          * редко имеют более низкий показатель, чем обычно, хотя спортивные лошади может иметь скорость 20-25
          * животные с болезненными поражениями или страдающие циркуляторным шоком может быть учащенное сердцебиение

  6: частота дыхания

          * линейный
          * нормальная скорость от 8 до 10
          * полезность сомнительна из-за больших колебаний

  7: температура конечностей

          * субъективное указание на периферическое кровообращение
          * возможные значения:
               1. = нормальный
               2. = теплый
               3. = круто
               4. = холодный
          * холодные или холодные конечности указывают на возможный шок
          * горячие конечности должны коррелировать с повышенной ректальной температурой.

  
 11: боль - субъективная оценка уровня боли лошади.

          * возможные значения:
               1. = тревога, боли нет
               2. = депрессия
               3. = перемежающаяся легкая боль
               4. = перемежающаяся сильная боль
               5. = постоянная сильная боль
          * НЕ следует рассматривать как упорядоченную или дискретную переменную!
          * В общем, чем больнее, тем больше вероятность, что потребуется операция
          * предшествующее лечение боли может в некоторой степени маскировать уровень боли

 23: исход
 
          * что в итоге случилось с лошадью?
          * возможные значения:
               1. = жил
               2. = умер
               3. = был усыплен


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
?pd.read_csv

In [ ]:

cols = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']
datahorse = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', usecols =[1-1, 2-1, 4-1, 5-1, 6-1, 7-1, 11-1, 23-1], names=cols, encoding='utf-8')
datahorse.head()

In [ ]:
datahorse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   surgery?                    300 non-null    object
 1   Age                         300 non-null    int64 
 2   rectal temperature          300 non-null    object
 3   pulse                       300 non-null    object
 4   respiratory rate            300 non-null    object
 5   temperature of extremities  300 non-null    object
 6   pain                        300 non-null    object
 7   outcome                     300 non-null    object
dtypes: int64(1), object(7)
memory usage: 18.9+ KB


In [ ]:
datahorse.describe()

,Age
count,300.000000
mean,1.640000
std,2.173972
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,9.000000


### Сейчас важно правильно прочесть файл!
Без `na_values = '?'` или `na_values = ['?']`
прочитывается как папало и НЕ выдает нужную инфу по ДФ


In [ ]:
# оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome
# 1,2,4,5,6,7,11,23

columns = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']
data_horse = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', usecols =[1-1, 2-1, 4-1, 5-1, 6-1, 7-1, 11-1, 23-1], names=columns, encoding='utf-8', na_values = '?')
data_horse.head()


In [ ]:
data_horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [ ]:
data_horse.describe()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [ ]:
data_horse['Age'].unique()

array([1, 9])

Странно.
По описанию должны быть цифры 1 и 2 (взрослая и молодая соответственно)

          1. 1 = взрослая лошадь
          2. 2 = Молодой (< 6 месяцев)

Почему мне показывает 1 и 9? 

Посмотрим, много ли таких 9?)

In [ ]:
# посчитаем, сколько раз встречается каждое из значений
data_horse['Age'].value_counts() 

1    276
9     24
Name: Age, dtype: int64

Вполне возможно, что это ошибка ввода данных. Возможно, человек опечатался. Нет опыта, не могу знать, возможно ли опечататься 24 раза?)) Сомнительно)). Но факт остается фактом, я здесь не вижу ни одной 2, зато вижу 24 раза 9

Раз все остальные цифры == 1, то, скорее всего, в этой конюшне только взрослые лошади.

Не знаю, великие ли это знания?)))
Я не могу сказать по этой инфе, что на ферме(табуне) жеребцы и(или) мерины)) С другой стороны, если бы были и кобылы в табуне, то были бы и жеребята, т.е отметка == 2)
Пока что -- полный мрак))

In [ ]:
# поизучаем данные)
type(data_horse['rectal temperature'])
print(pd.to_numeric(data_horse['rectal temperature'], errors='coerce').max())
print(pd.to_numeric(data_horse['rectal temperature'], errors='coerce').min())
print(pd.to_numeric(data_horse['rectal temperature'], errors='coerce').median())
# data_horse['surgery?'].astype(int)


40.8
35.4
38.2


In [ ]:
# Смотрю данные по ректальной температуре.
print(len(data_horse['rectal temperature'].unique()))
print(data_horse['rectal temperature'].unique())
print(data_horse['rectal temperature'].mode()[0])
print(data_horse['rectal temperature'].median())
data_horse['rectal temperature'].value_counts()

41
[38.5 39.2 38.3 39.1 37.3  nan 37.9 38.1 37.2 38.  38.2 37.6 37.5 39.4
 39.9 38.4 38.6 37.8 37.7 39.  35.4 38.9 37.4 40.3 37.  39.7 38.7 36.4
 38.8 39.6 36.8 39.5 36.5 36.  37.1 39.3 36.1 36.6 40.8 36.9 40. ]
38.0
38.2


38.0    26
38.5    19
38.3    18
37.8    17
38.2    16
38.1    12
37.5    12
38.6    12
38.4    11
37.9     7
37.2     7
37.6     7
37.7     7
38.7     7
37.3     6
38.8     6
39.2     5
39.5     4
39.3     4
38.9     4
39.0     4
39.4     3
37.1     3
37.4     3
40.3     2
37.0     2
39.1     2
36.5     2
36.9     1
40.8     1
36.6     1
36.1     1
35.4     1
36.0     1
36.8     1
39.6     1
39.9     1
36.4     1
39.7     1
40.0     1
Name: rectal temperature, dtype: int64

In [ ]:
# смотрим, много ли лошадей с повышенной температурой
higt_temper = data_horse.loc[data_horse['rectal temperature'] > 38.7]

In [ ]:
print(higt_temper['rectal temperature'].count())
higt_temper['rectal temperature'].value_counts()

39


38.8    6
39.2    5
39.0    4
38.9    4
39.5    4
39.3    4
39.4    3
39.1    2
40.3    2
39.9    1
39.7    1
39.6    1
40.8    1
40.0    1
Name: rectal temperature, dtype: int64

Ладно, накинем 1 градус сверх нормы -- будем считать, что температура поднимается после физических нагрузок.
 
Но, у нас есть 39 лошадок, сверх 38.7 градусов. Полагаю, это ллоашадки болеют

Никаких аномальных показателей я не вижу. 

НО так смотреть совсем не удобно. Раскидаю наших лошадок по категориям, согласно этого показателя. Так смогу быстрее оценить положение по нашей конюшне)

In [ ]:
# data_horse['category of rectal temperature'] = np.where(data_horse['rectal temperature'] < 38.7)

In [ ]:
def rectal_tem_filter(row):
    """
    Фильтрует строки по праивлам:
    если температуры до 38.7 -- в пределах нормы
    38.7-39.7 -- больна
    39.7 -- критическое состояние
    """
    rect_category = row['rectal temperature']

    if rect_category < 38.7:
        return 'normal'
    elif 38.7 <= rect_category < 39.7:
        return 'sick'
    elif rect_category >= 39.7:
        return 'critical'

    return rect_category

In [ ]:
data_horse['category of rectal temperature'] = data_horse.apply(rectal_tem_filter, axis=1)
data_horse

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0,normal
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0,sick
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0,normal
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0,normal
...,...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0,NaN
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0,normal
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0,normal
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0,normal


In [ ]:
print(data_horse['category of rectal temperature'].count())
data_horse['category of rectal temperature'].value_counts()

240


normal      194
sick         40
critical      6
Name: category of rectal temperature, dtype: int64

In [ ]:
print(46/240)
print(60/300)
print(46/300)

0.19166666666666668
0.2
0.15333333333333332


У нас нет данных по 20% лашадок ( должно быть 300, а есть только по 240)
Ладно, будем считать, что измеряли температуру только после физнагрузки, чтоб понять, насколько загнана лошадь, или при подозрении на болезнь.

Из тех, кого измеряли почти 5-я часть больна и(или) в критическом состоянии

Если принять, что 20 не попавших в отчет -- здоровы, то получаем из 300 лошадок 15.3% в плохом состоянии.


ПОКА я ничего не могу сказать, посмотрим, что найдем ещё

In [ ]:
# 30-40 нормально для взрослых
# * редко имеют более низкий показатель, чем обычно, хотя спортивные лошади может иметь скорость 20-25

print(len(data_horse['pulse'].unique()))
print(data_horse['pulse'].unique())
print(data_horse['pulse'].mode()[0])
print(data_horse['pulse'].median())
print(data_horse['pulse'].min())
print(data_horse['pulse'].max())

53
[ 66.  88.  40. 164. 104.  nan  48.  60.  80.  90.  72.  42.  92.  76.
  96. 128.  64. 110. 130. 108. 100. 112.  52. 146. 150. 120. 140.  84.
  46. 114. 160.  54.  56.  38.  98.  50.  44.  70.  78.  49.  30. 136.
 132.  65.  86. 129.  68.  36.  45. 124. 184.  75.  82.]
48.0
64.0
30.0
184.0


Чаще всего встречаются лошади с пульсом 48. Половна из лошадок имеют пульс больше 64.
Ладно, тут длина уникальных значений всего 53, я могу видеть и значения и 160+, и 180.

Могут ли лошади в нормальном состоянии иметь пульс в 3-4 раза больше нормы? И насколько эти показатели показывают стабильность такого пульса и как продолжительно?

Попробую и здесь разделить на категории на свой взгляд

In [ ]:
def pulse_filter(row):
    """
    Фильтрует строки по правилам:
    если pulse до 40*2 -- в пределах нормы
    40*2-40*3 -- угроза
    более 40*3-- критическое состояние
    """
    pulse_category = row['pulse']

    if pulse_category < 40*2:
        return 'normal'
    elif 40*2 <= pulse_category < 40*3:
        return 'danger'
    elif pulse_category >= 40*3:
        return 'critical'

    return pulse_category

In [ ]:
data_horse['category of pulse'] = data_horse.apply(pulse_filter, axis=1)
data_horse

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0,normal,normal
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0,sick,danger
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0,normal,normal
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick,critical
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0,normal,danger
...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0,NaN,critical
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0,normal,normal
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0,normal,normal
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0,normal,danger


In [ ]:
print(data_horse['category of pulse'].count())
data_horse['category of pulse'].value_counts()

276


normal      177
danger       73
critical     26
Name: category of pulse, dtype: int64

In [ ]:
# посмотрю, как много лошадок в плохом состоянии в общей массе
print(26/276) 
print((73+26)/276)
print(26/300)
print((73+26)/300)

print('------------------')
# посмотрю, во сколько раз максимум больше моды и медианы
print(data_horse['pulse'].max()/data_horse['pulse'].mode()[0])
print(data_horse['pulse'].max()/data_horse['pulse'].median())

0.09420289855072464
0.358695652173913
0.08666666666666667
0.33
------------------
3.8333333333333335
2.875


Опять-таки буду считать,что 24 лошадки не попали в общий список, потому что не вызывают никакого беспокойста, т.е абсолютно здоровы и счастливы.

Но всё равно, цифры не совсем утешительны. Лошадей в критическом состоянии пульса почит 9% из всё массы -- 300 лошадей.
А если в зону риска внести критических и опасная зона, то получим больше трети из всей массы лошадей -- 300.

Если 9-30% лошадей больны, и больны например инфекционной болезнью, то ферма в большом риске получить тотальную эпидемию

In [ ]:
# посмотрю, во сколько раз максимум больше моды и медианы
print(data_horse['pulse'].max()/data_horse['pulse'].mode()[0])
print(data_horse['pulse'].max()/data_horse['pulse'].median())

3.8333333333333335
2.875


In [ ]:
# СКО до заполнения пустот
print(data_horse['rectal temperature'].std())
print(np.std(data_horse['rectal temperature'], ddof=1))
print('---------')
print(data_horse['pulse'].std())
print(np.std(data_horse['pulse'], ddof=1))

0.7322886641121578
0.7322886641121578
---------
28.630556660735003
28.630556660735003


In [ ]:
# Дисперсия до заполнения пустот
print(np.var(data_horse['rectal temperature'], ddof=1))
print(data_horse['rectal temperature'].var())
print('---------')
print(np.var(data_horse['pulse'], ddof=1))
print(data_horse['pulse'].var())

0.5362466875871686
0.5362466875871686
---------
819.7087747035575
819.7087747035575


In [ ]:
# Теперь уберем пустоты в нужных колонках
# сначала уберем все пустоты из колонки 'rectal temperature'
dh_rt_0 = data_horse[~data_horse['rectal temperature'].isnull()]
# dh_rt_0.info()

# Теперь уберем пустоты из колонки 'pulse'
dh_rt_p_0 = dh_rt_0[~dh_rt_0['pulse'].isnull()]
dh_rt_p_0.info()

In [ ]:
# СКО после удаления пустот
print(dh_rt_p_0['rectal temperature'].std())
print(np.std(dh_rt_p_0['rectal temperature'], ddof=1))
print('--------------')
print(dh_rt_p_0['pulse'].std())
print(np.std(dh_rt_p_0['pulse'], ddof=1))


0.7284645844147041
0.7284645844147041
--------------
28.935036950086776
28.935036950086776


Да, изменилось после удаления пустот, но не критически

In [ ]:
# Дисперсия после удаления пустот
print(np.var(dh_rt_p_0['rectal temperature'], ddof=1))
print(dh_rt_p_0['rectal temperature'].var())
print('--------------')
print(np.var(dh_rt_p_0['pulse'], ddof=1))
print(dh_rt_p_0['pulse'].var())


0.5306606507464875
0.5306606507464875
--------------
837.236363302887
837.236363302887


По пульсу дисперсия изменилась

Выбросы. 
По определению (если можно так сказать))), выбросы являются нетипичными, резко выделяющимися наблюдениями. 

Смотрю, в целом у лошадей пульс выше определённой доком нормы: 20-25 или 30-40. У большинства пульс в пределах допустимого.
но есть почти 9% лошадей, у которых пульс в 3 раза больше нормы. Боюсь сильно ошибиться, но по моему мнению, это противеречит природе. 

Совсем нет опыта, но неужели выбросы могут составлять 9%? Не много ли это, учитывая, что цифры возводятся в квадрат, что может существенно исказить анализ?

Попробую межквартильный размах посчитать и уже потом посмотреть как наши 9% выглядят на фоне IQR?)

In [ ]:
# Ищем выбросы, исключив пустоты
# IQR без пустот
Q1_pls = dh_rt_p_0['pulse'].quantile(0.25)
Q3_pls = dh_rt_p_0['pulse'].quantile(0.75)
IQR_pls = Q3_pls - Q1_pls
IQR_pls

# Диапазон, куда выбросы НЕ входят
lower_bound_pls = Q1_pls - (1.5 * IQR_pls) 
upper_bound_pls = Q3_pls + (1.5 * IQR_pls)

# Есть ли эти самые выбросы?
print(len(dh_rt_p_0[~dh_rt_p_0['pulse'].between(lower_bound_pls, upper_bound_pls, inclusive=True)]))

5


<ipython-input-15-b1ccbf92bae2>:13: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  print(len(dh_rt_p_0[~dh_rt_p_0['pulse'].between(lower_bound_pls, upper_bound_pls, inclusive=True)]))


In [ ]:
# Посмотрим, что это за выбросы?
dh_rt_p_0[~dh_rt_p_0['pulse'].between(lower_bound_pls, upper_bound_pls, inclusive=True)]

<ipython-input-16-4c1e84346a03>:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  dh_rt_p_0[~dh_rt_p_0['pulse'].between(lower_bound_pls, upper_bound_pls, inclusive=True)]


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick,critical
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0,sick,critical
55,1.0,9,38.6,160.0,20.0,3.0,3.0,2.0,normal,critical
255,1.0,9,38.8,184.0,84.0,1.0,4.0,2.0,sick,critical
275,1.0,9,38.8,150.0,50.0,1.0,5.0,2.0,sick,critical


Оки, вижу, что попало то, что выше 40*3) Значит, я была близка в своих рассуждениях о возможности такого) Ура!))


Теперь просто для сравнения посчитаю всё тоже самое, но с пропусками

In [ ]:
# IQR с пустотами
Q1_pulse = data_horse['pulse'].quantile(0.25)
Q3_pulse = data_horse['pulse'].quantile(0.75)
IQR_pulse = Q3_pulse - Q1_pulse
IQR_pulse

40.0

In [ ]:
# Диапазон, куда выбросы НЕ входят
lower_bound_pulse = Q1_pulse - (1.5 * IQR_pulse) 
upper_bound_pulse = Q3_pulse + (1.5 * IQR_pulse)

In [ ]:
# Есть ли эти самые выбросы?
print(len(data_horse[~data_horse['pulse'].between(lower_bound_pulse, upper_bound_pulse, inclusive=True)]))

In [ ]:
# что это за выбросы?
data_horse[~data_horse['pulse'].between(lower_bound_pulse, upper_bound_pulse, inclusive=True)]

Оки, вижу, что попало то, что выше 40*3) Значит, я была близка в своих рассуждениях о возможности такого) Ура!))

И вижу, что сюда прилетело всё с пропусками. Оки, посмотрим на это же после заполнения пропусков.


  6: частота дыхания

          * линейный
          * нормальная скорость от 8 до 10
          * полезность сомнительна из-за больших колебаний

Смотрим это тоже)

In [ ]:
# Ищем выбросы, исключив пустоты
# IQR без пустот
Q1_rctl = dh_rt_p_0['rectal temperature'].quantile(0.25)
Q3_rctl = dh_rt_p_0['rectal temperature'].quantile(0.75)
IQR_rctl = Q3_rctl - Q1_rctl
IQR_rctl

# Диапазон, куда выбросы НЕ входят
lower_bound_rctl = Q1_rctl - (1.5 * IQR_rctl) 
upper_bound_rctl = Q3_rctl + (1.5 * IQR_rctl)

# Есть ли эти самые выбросы?
print(len(dh_rt_p_0[~dh_rt_p_0['rectal temperature'].between(lower_bound_rctl, upper_bound_rctl, inclusive=True)]))

14


<ipython-input-20-1d00149ee44d>:13: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  print(len(dh_rt_p_0[~dh_rt_p_0['rectal temperature'].between(lower_bound_rctl, upper_bound_rctl, inclusive=True)]))


In [ ]:
# Посмотрим, что это за выбросы?
dh_rt_p_0[~dh_rt_p_0['rectal temperature'].between(lower_bound_rctl, upper_bound_rctl, inclusive=True)]

<ipython-input-21-b9225bbf8eb9>:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  dh_rt_p_0[~dh_rt_p_0['rectal temperature'].between(lower_bound_rctl, upper_bound_rctl, inclusive=True)]


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0,critical,normal
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0,normal,critical
54,2.0,1,40.3,114.0,36.0,3.0,2.0,3.0,critical,danger
75,1.0,9,39.7,100.0,NaN,3.0,2.0,3.0,critical,danger
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0,normal,danger
91,2.0,1,40.3,114.0,36.0,3.0,2.0,2.0,critical,danger
99,2.0,1,39.6,108.0,51.0,3.0,2.0,1.0,sick,danger
118,1.0,1,36.5,78.0,30.0,1.0,5.0,1.0,normal,normal
141,2.0,1,36.0,42.0,30.0,NaN,NaN,2.0,normal,normal
238,2.0,1,36.1,88.0,NaN,3.0,3.0,3.0,normal,danger


In [ ]:
# IQR ректальной температуры
Q1_rectal = data_horse['rectal temperature'].quantile(0.25)
Q3_rectal = data_horse['rectal temperature'].quantile(0.75)
IQR_rectal = Q3_rectal - Q1_rectal
print(IQR_rectal)

# IQR частоты дыхания
Q1_respiratore = data_horse['respiratory rate'].quantile(0.25)
Q3_respiratore = data_horse['respiratory rate'].quantile(0.75)
IQR_respiratore = Q3_respiratore - Q1_respiratore
print(IQR_respiratore)

0.7000000000000028
17.5


In [ ]:
# Диапазон, куда выбросы НЕ входят данные по ректальной t
lower_bound_rectal = Q1_rectal - (1.5 * IQR_rectal) 
upper_bound_rectal = Q3_rectal + (1.5 * IQR_rectal)

# Диапазон, куда выбросы НЕ входят данные по дыханию
lower_bound_respiratore = Q1_respiratore - (1.5 * IQR_respiratore) 
upper_bound_respiratore = Q3_respiratore + (1.5 * IQR_respiratore)

In [ ]:
# Есть ли эти самые выбросы по ректальной t?
print(len(data_horse[~data_horse['rectal temperature'].between(lower_bound_rectal, upper_bound_rectal, inclusive=True)]))

# Есть ли эти самые выбросы по дыханию?
print(len(data_horse[~data_horse['respiratory rate'].between(lower_bound_respiratore, upper_bound_respiratore, inclusive=True)]))

74
75


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """


Ого!!! 74 и 75 из 300 выбросы! А я переживала про 9%))

In [ ]:
# что это за выбросы по рект t?
data_horse[~data_horse['rectal temperature'].between(lower_bound_rectal, upper_bound_rectal, inclusive=True)]

# что это за выбросы по дыханию?
data_horse[~data_horse['respiratory rate'].between(lower_bound_respiratore, upper_bound_respiratore, inclusive=True)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick,critical
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0,NaN,NaN
7,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0,NaN,normal
9,2.0,9,38.3,90.0,NaN,1.0,5.0,1.0,normal,danger
28,1.0,1,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
274,1.0,1,NaN,76.0,NaN,NaN,NaN,3.0,NaN,normal
281,2.0,1,40.0,78.0,NaN,3.0,2.0,2.0,critical,normal
284,2.0,1,38.5,54.0,NaN,1.0,3.0,1.0,normal,normal
288,1.0,1,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


Отлично! в Выбросы (до заполнения пустот) попали лошадки с высокой температурой и с NAN

 В выбросы по дыханию попали лошадки с частотой дыхания в 7-8+ раз больше нормы))

Кстати, сюда жа вошла лошадка, которую я отметила, как критическую по пульсу. И эта же лошадка отмечена, как мёртвая. Оно и понятно теперь))

Попробую понять, насколько связаны выброс пульса, температуры и дыхания с тем, что лошадка умерла и(или) её усыпили

In [ ]:
print(data_horse.groupby('outcome')['pulse'].median())
# print(data_horse.groupby('outcome')['pulse'].mode()[0]) # Error :(

print(data_horse.groupby('outcome')['respiratory rate'].median())
# print(data_horse.groupby('outcome')['respiratory rate'].mode()[0])

print(data_horse.groupby('outcome')['rectal temperature'].median())

outcome
1.0    54.0
2.0    86.0
3.0    82.0
Name: pulse, dtype: float64
outcome
1.0    24.0
2.0    30.0
3.0    24.0
Name: respiratory rate, dtype: float64
outcome
1.0    38.20
2.0    38.10
3.0    38.05
Name: rectal temperature, dtype: float64


Ну, и что я получила?)) Что только по пульсу более-менее можно понять, как заполнить пропуски)

Значит, остальное заполним медианой. Не буду заполнять средним, так как много выбросов нашла по всем столбцам. А они искажают среднее.

Возможно, следовало бы заполнять пропуски в несколько этапов:
1. сначала по пульсу и исходу заполнить пульс.
2. опять сгруппировать по исходу, пульсу -- заполнить дыхание
3. сгруппировать по пульсу, исходу, дыханию  -- заполнить температуру.

Попробую заморочиться)

In [ ]:
data_horse_test = data_horse.copy()
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              240 non-null    float64
 3   pulse                           276 non-null    float64
 4   respiratory rate                242 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         299 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
data_horse_test['outcome'] = data_horse_test['outcome'].fillna(data_horse_test['outcome'].mode()[0])

In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              240 non-null    float64
 3   pulse                           276 non-null    float64
 4   respiratory rate                242 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
print(data_horse_test.groupby('outcome')['pulse'].median())

outcome
1.0    54.0
2.0    86.0
3.0    82.0
Name: pulse, dtype: float64


для заполнение модой через transform можно использовать вот такой синтаксис (из-за того, что mode() возвращает массив): 

df['col'].fillna(df.groupby('group_col')['col'].transform(lambda x: x.mode()[0]))

, это было бы более уместно для заполнения категориальных значений.

In [ ]:
data_horse_test['pulse'] = data_horse_test['pulse'].fillna(data_horse_test.groupby('outcome')['pulse'].transform(lambda x: x.mode()[0]))

In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              240 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                242 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
data_horse_test

In [ ]:
print(data_horse_test.groupby(['outcome', 'pulse'])['respiratory rate'].median())

outcome  pulse
1.0      30.0     18.0
         36.0      9.0
         38.0     20.0
         40.0     16.0
         42.0     26.0
                  ... 
3.0      112.0    16.0
         114.0    36.0
         120.0    59.0
         130.0    16.0
         140.0    24.0
Name: respiratory rate, Length: 98, dtype: float64


Интересно, почему нет исхода == 2?

In [ ]:
data_horse_test['respiratory rate'] = data_horse_test['respiratory rate'].fillna(data_horse_test.groupby(['outcome', 'pulse'])['respiratory rate'].transform('median'))

In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              240 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                292 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


Не все строки 'respiratory rate' заполнились.

Заполню медианой или модой

In [ ]:
data_horse_test['respiratory rate'] = data_horse_test['respiratory rate'].fillna(data_horse_test['respiratory rate'].median())

In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              262 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                300 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
print(data_horse_test.groupby(['outcome', 'pulse', 'respiratory rate'])['rectal temperature'].median())

outcome  pulse  respiratory rate
1.0      30.0   18.0                38.5
         36.0   9.0                 37.2
         38.0   20.0                37.6
         40.0   8.0                 37.1
                12.0                37.8
                                    ... 
3.0      120.0  48.0                39.4
                59.0                 NaN
                70.0                 NaN
         130.0  16.0                38.2
         140.0  24.0                35.4
Name: rectal temperature, Length: 218, dtype: float64


In [ ]:
data_horse_test['rectal temperature'] = data_horse_test['rectal temperature'].fillna(data_horse_test.groupby(['outcome', 'pulse', 'respiratory rate'])['rectal temperature'].transform('median'))

In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              262 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                300 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


Остались пустые строки.

Заполню их median, так как разброс температуры не тако огромный, как пульса

In [ ]:
data_horse_test['rectal temperature'] = data_horse_test['rectal temperature'].fillna(data_horse_test['rectal temperature'].median())

In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              300 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                300 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               276 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


Обновим наши строки по категориям 'category of rectal temperature' и 'category of pulse', чтоб убрать и там пустоты          

In [ ]:
data_horse_test['category of pulse'] = data_horse_test.apply(pulse_filter, axis=1)
data_horse_test

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0,normal,normal
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0,sick,danger
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0,normal,normal
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick,critical
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0,normal,danger
...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0,NaN,critical
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0,normal,normal
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0,normal,normal
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0,normal,danger


In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              300 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                300 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  240 non-null    object 
 9   category of pulse               300 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
data_horse_test['category of rectal temperature'] = data_horse_test.apply(rectal_tem_filter, axis=1)
data_horse_test

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0,normal,normal
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0,sick,danger
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0,normal,normal
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick,critical
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0,normal,danger
...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0,normal,critical
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0,normal,normal
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0,normal,normal
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0,normal,danger


In [ ]:
data_horse_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   surgery?                        299 non-null    float64
 1   Age                             300 non-null    int64  
 2   rectal temperature              300 non-null    float64
 3   pulse                           300 non-null    float64
 4   respiratory rate                300 non-null    float64
 5   temperature of extremities      244 non-null    float64
 6   pain                            245 non-null    float64
 7   outcome                         300 non-null    float64
 8   category of rectal temperature  300 non-null    object 
 9   category of pulse               300 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
data_horse_test[['category of rectal temperature', 'category of pulse']].value_counts()

category of rectal temperature  category of pulse
normal                          normal               168
                                danger                69
sick                            normal                18
normal                          critical              17
sick                            danger                13
                                critical               9
critical                        danger                 3
                                normal                 3
dtype: int64

In [ ]:
# IQR пульсу
Q1_pulse_test = data_horse_test['pulse'].quantile(0.25)
Q3_pulse_test = data_horse_test['pulse'].quantile(0.75)
IQR_pulse_test = Q3_pulse_test - Q1_pulse_test
print(IQR_pulse_test)

# IQR ректальной температуры
Q1_rectal_test = data_horse_test['rectal temperature'].quantile(0.25)
Q3_rectal_test = data_horse_test['rectal temperature'].quantile(0.75)
IQR_rectal_test = Q3_rectal_test - Q1_rectal_test
print(IQR_rectal_test)

# IQR частоты дыхания
Q1_respiratore_test = data_horse_test['respiratory rate'].quantile(0.25)
Q3_respiratore_test = data_horse_test['respiratory rate'].quantile(0.75)
IQR_respiratore_test = Q3_respiratore_test - Q1_respiratore_test
print(IQR_respiratore_test)

# было до заполнения пустот:
# 40.0
# 0.7000000000000028
# 17.5

40.0
0.7000000000000028
16.0


In [ ]:
# Диапазон, куда выбросы НЕ входят данные по пульсу
lower_bound_pulse_test = Q1_pulse_test - (1.5 * IQR_pulse_test) 
upper_bound_pulse_test = Q3_pulse_test + (1.5 * IQR_pulse_test)

# Диапазон, куда выбросы НЕ входят данные по ректальной t
lower_bound_rectal_test = Q1_rectal_test - (1.5 * IQR_rectal_test) 
upper_bound_rectal_test = Q3_rectal_test + (1.5 * IQR_rectal_test)

# Диапазон, куда выбросы НЕ входят данные по дыханию
lower_bound_respiratore_test = Q1_respiratore_test - (1.5 * IQR_respiratore_test) 
upper_bound_respiratore_test = Q3_respiratore_test + (1.5 * IQR_respiratore_test)

In [ ]:
# что это за выбросы по рект t?
print(len(data_horse_test[~data_horse_test['pulse'].between(lower_bound_pulse_test, upper_bound_pulse_test, inclusive=True)]))

# что это за выбросы по рект t?
print(len(data_horse_test[~data_horse_test['rectal temperature'].between(lower_bound_rectal_test, upper_bound_rectal_test, inclusive=True)]))

# что это за выбросы по дыханию?
print(len(data_horse_test[~data_horse_test['respiratory rate'].between(lower_bound_respiratore_test, upper_bound_respiratore_test, inclusive=True)]))

5
14
17


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  


Вижу, что количество строк, считаемые выбросами, занчительно уменьшилось.

Было:

29

74

75

In [ ]:
data_horse_test[~data_horse_test['pulse'].between(lower_bound_pulse_test, upper_bound_pulse_test, inclusive=True)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """Entry point for launching an IPython kernel.


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome,category of rectal temperature,category of pulse
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0,sick,critical
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0,sick,critical
55,1.0,9,38.6,160.0,20.0,3.0,3.0,2.0,normal,critical
255,1.0,9,38.8,184.0,84.0,1.0,4.0,2.0,sick,critical
275,1.0,9,38.8,150.0,50.0,1.0,5.0,2.0,sick,critical


Что я вижу?

Я вижу, что в эти выбросы попали лошадки с возрастом 9, которого не должно быть в априори.
Сказала бы, что всё хорошо, с этими данными, так как вижу, что и дыханием у этих лошадок не всё хорошо, но ижу, что попала одна лошадь с нормальным -- 20 -- дыханием.

Хотя, если судить по "этим выбросам", то получается, что лошадка № 41 одна была без операции и в итоге выжившая)) Остальные не пережили операции, видимо)



Сравним средние показатели до заполнения пустот и после заполнения БЕЗ учета этих 5 строк

In [ ]:
len(data_horse_test[data_horse_test['pulse'].between(lower_bound_pulse_test, upper_bound_pulse_test, inclusive=True)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """Entry point for launching an IPython kernel.


295

In [ ]:
data_horse.describe()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [ ]:
data_horse_test[data_horse_test['pulse'].between(lower_bound_pulse_test, upper_bound_pulse_test, inclusive=True)].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  """Entry point for launching an IPython kernel.


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,294.000000,295.000000,295.000000,295.000000,295.000000,240.000000,241.000000,295.000000
mean,1.401361,1.515254,38.159492,70.223729,29.288136,2.350000,2.941909,1.545763
std,0.491009,1.967146,0.659090,25.402557,15.646011,1.040196,1.308922,0.740317
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.450000,87.000000,35.500000,3.000000,4.000000,2.000000
max,2.000000,9.000000,40.800000,146.000000,96.000000,4.000000,5.000000,3.000000


Я не вижу критических изменений)

Да, std изменилось. Это понятно) там квадраты.

В средних, медианах, квартилях изменилось совсем немного.

Максимум по поульсу ушёл после исключения выбросов

Теперь посмотрю, какая из категорий встречается больше всего по пульсу и по исходу

In [ ]:
data_horse_test['category of pulse'].mode()[0]

'normal'

In [ ]:
data_horse_test['outcome'].mode()[0]

1.0

In [ ]:
data_horse_test['category of pulse'].value_counts()

normal      189
danger       85
critical     26
Name: category of pulse, dtype: int64

In [ ]:
data_horse_test['outcome'].value_counts()

1.0    179
2.0     77
3.0     44
Name: outcome, dtype: int64

Вижу, что большинство лошадок живы и в нормальном статусе. Однако больше трети всё же в зонах danger и critical.

Значит, содержатся в приемлимых физических условиях. Но, возможно, не в совсем нормальных психологических условиях. Возможно, это повышает пульс? Возможно, лошадей замеряли после скачек?

И по статистике исход видно, что больше трети так или иначе уже нет в живых. 

Глупо плохо содержать животных, на которых собираешься зарабатывать.

Лошадку нужно вырастить, натренировать, чтоб она приносила дивиденты. И плюс она должна быть всегда здорова физически и пчихически. чтоб хорошо выступать.
Иначе, ферма будет убыточна.

Не могу предположить, что ферма не скаковая. В наше время конина совсем не популярна. А содержать коней сложно и затратно. Не думаю, что колбасы окупают всё) Хотя, "Есть многое на свете, друг Горацио, что и не снилось нашим мудрецам"

Не могу сказать ничего определенного, кроме того, что летальный исход на ферме явление не из ряда вон выходящее.


ПС не знаю, уместна ли здесь эта моя мысль?)))
И, конечно же, не стоит выбрасывать из головы, что данные в эти таблицы могли заноситься небрежно, если вспомнить "Возраст" == 9 и значения пульса, превышающие норму в 4 раза.

#Полезные ссылки

[Colab с вебинара](https://colab.research.google.com/drive/1gQq1IVkXJFh2NTmkFcF5WSkgsKbI6F8S#scrollTo=0JuWO8oMOU5g)

[Pandas read_csv() with Examples](https://sparkbyexamples.com/pandas/pandas-read-csv-examples/)

[Важность правильного прочтения файла!](https://www.geeksforgeeks.org/use-of-na_values-parameter-in-read_csv-function-of-pandas-in-python/)

Без `na_values = '?'` или `na_values = ['?']` прочитывается как папало и НЕ выдает нужную инфу по ДФ

[Ещё про CSV файлы и работе с ними](http://net-informations.com/ds/pda/csv.htm)

[Изменить тип данных столбцов в Pandas](https://www.delftstack.com/ru/howto/python-pandas/how-to-change-data-type-of-columns-in-pandas/)

[Еще про выбросы](http://statsoft.ru/home/textbook/modules/stbasic.html#Correlationse)

[How to select rows with NaN in particular column?](https://stackoverflow.com/questions/43831539/how-to-select-rows-with-nan-in-particular-column)